### Package Imports

In [1]:
from time import time
init = time()

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sys
import geopandas as gpd
import warnings
warnings.filterwarnings("ignore")

### Data loading and preprocessing

In [2]:
df = pd.read_csv("../../data/pm.csv.gz")
df = df[['Date', 'Site ID', 'Daily Mean PM10 Concentration', 'PERCENT_COMPLETE', 'STATE_CODE', 'STATE', 'COUNTY_CODE', 
         'COUNTY', 'SITE_LATITUDE', 'SITE_LONGITUDE']]
df = df[df['STATE']!='Hawaii']
df = df[df['STATE']!='Alaska']
df = df[df['STATE']!='New Mexico']

t1 = df[(df['SITE_LONGITUDE'] < -104.01) | (df['SITE_LONGITUDE']>-99.42)]

t1.head(2)

,Date,Site ID,Daily Mean PM10 Concentration,PERCENT_COMPLETE,STATE_CODE,STATE,COUNTY_CODE,COUNTY,SITE_LATITUDE,SITE_LONGITUDE
0,01/11/1990,120111003,45,100.0,12,Florida,11,Broward,26.129531,-80.168379
1,01/17/1990,120111003,21,100.0,12,Florida,11,Broward,26.129531,-80.168379


### DataFrame to store results

In [3]:
res_df = pd.DataFrame(columns=[ 'mean', '25%', '50%', '75%'])

### Result: Monitored Anually average PM10

In [4]:
def summary_stats(x):
    return x.mean(), np.percentile(x, 25), np.percentile(x, 50), np.percentile(x, 75)

In [5]:
ann_df = t1.groupby('Site ID').mean().reset_index()
res_df.loc['AA_Mon', :] = summary_stats(ann_df['Daily Mean PM10 Concentration'].values)
res_df

,mean,25%,50%,75%
AA_Mon,28.943658,22.391667,27.299806,33.634208


### Define data for interpolation

In [6]:
X = ann_df[['SITE_LONGITUDE', 'SITE_LATITUDE']].values
y = ann_df[['Daily Mean PM10 Concentration']].values.ravel()

### Spatial Interpolation

In [7]:
from polire import SpatialAverage
spatial = SpatialAverage(coordinate_type='Geographic', radius=16.1)
spatial.fit(X, y)

SpatialAverage

In [8]:
centroids = np.load("../../data/block_groups/usa_centroids.npy")

spatial_predictions = spatial.predict(centroids)
spatial_predictions = spatial_predictions[~np.isnan(spatial_predictions)]

In [9]:
res_df.loc['AA_SA', :] = summary_stats(spatial_predictions)
res_df

,mean,25%,50%,75%
AA_Mon,28.943658,22.391667,27.299806,33.634208
AA_SA,31.944817,25.833431,30.178879,34.733065


### IDW

In [10]:
from polire import IDW
from polire.utils.distance import haversine, euclidean

idw = IDW(coordinate_type='Geographic')
idw.fit(X, y)

idw_predictions = idw.predict(centroids)

In [11]:
res_df.loc['AA_IDW', :] = summary_stats(idw_predictions)
res_df

,mean,25%,50%,75%
AA_Mon,28.943658,22.391667,27.299806,33.634208
AA_SA,31.944817,25.833431,30.178879,34.733065
AA_IDW,30.818663,26.915439,29.427778,32.42217


# $k$-NN


In [12]:
from polire import CustomInterpolator
from sklearn.neighbors import KNeighborsRegressor
knn = CustomInterpolator(KNeighborsRegressor(n_neighbors=1))
knn.fit(X, y)
knn_predictions = knn.predict(centroids)

In [13]:
res_df.loc['AA_KNN', :] = summary_stats(knn_predictions)
res_df

,mean,25%,50%,75%
AA_Mon,28.943658,22.391667,27.299806,33.634208
AA_SA,31.944817,25.833431,30.178879,34.733065
AA_IDW,30.818663,26.915439,29.427778,32.42217
AA_KNN,30.337855,24.508772,28.27451,34.074074


### DATA VALIDATION

In [14]:
def func_select_11(df):
    t = df.groupby(['Date', 'Site ID', 'STATE', 'COUNTY_CODE','COUNTY']).mean()
    t = t.reset_index(None)
    t1 = t.groupby('Site ID').count()
    t11 = t.set_index(['Site ID'])
    t11 = t11[t1['Date']>11]
    t11 = t11.reset_index(None)
    return t11

### Creating region-wise dataframes

In [15]:
# North West
nw = ['Washington', 'Oregon', 'Wyoming','Idaho', 'Montana', 'Colorado','Utah','Nevada']
# South California
sc = ['California']
# Mid West
mw = ['West Virginia', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Michigan', 'Minnesota', 
      'Missouri', 'Nebraska', 'North Dakota', 'Ohio', 'South Dakota', 'Wisconsin'] 
# South East
se = ['Oklahoma','Florida', 'Alabama', 'Georgia',  'Tennessee', 'South Carolina', 'North Carolina', 
      'Virginia', 'Louisiana', 'Arkansas', 'Texas', 'Kentucky']
# North East
ne = ['Connecticut', 'Delaware', 'Maine', 'Maryland', 'Massachusetts', 'New Hampshire', 'New Jersey', 
      'New York', 'Pennsylvania', 'Rhode Island', 'Vermont','District Of Columbia']

nwdf = t1[np.isin(t1['STATE'], nw)]
mwdf = t1[np.isin(t1['STATE'], mw)]
sedf = t1[np.isin(t1['STATE'], se)]
nedf = t1[np.isin(t1['STATE'], ne)]
cal = t1[np.isin(t1['STATE'], ['California'])]
ncal = cal[cal['SITE_LATITUDE']>=34.81]
scal = cal[cal['SITE_LATITUDE']<34.81]
# nwdf = pd.concat([nwdf, t1[t1[np.isin(t1['STATE'], ['California'])]]['SITE_LATITUDE']>34.81])

nwdf = pd.concat([nwdf, ncal])

### Define quarterly dataframes 

In [16]:
mwq1 = mwdf[mwdf['Date']<='03/31/1990']
mwq2 = mwdf[(mwdf['Date']>='04/01/1990').values & (mwdf['Date']<='06/30/1990').values]
mwq3 = mwdf[(mwdf['Date']>='07/01/1990').values & (mwdf['Date']<='09/30/1990').values]
mwq4 = mwdf[(mwdf['Date']>='10/01/1990').values & (mwdf['Date']<='12/31/1990').values]

### Q1, Q2, Q3, Q4

In [17]:
def get_Xy(t):
    q = func_select_11(t)
    q.drop(columns=['STATE', 'COUNTY_CODE', 'COUNTY', 'PERCENT_COMPLETE'], inplace=True)
    q = q.groupby('Site ID').mean()
    X = q[['SITE_LONGITUDE', 'SITE_LATITUDE']].values
    y = q[['Daily Mean PM10 Concentration']].values
    return X, y.ravel()

t_list = [t1[t1['Date']<='03/31/1990'], 
          t1[(t1['Date']>='04/01/1990') & (t1['Date']<='06/30/1990')],
          t1[(t1['Date']>='07/01/1990') & (t1['Date']<='09/30/1990')],
          t1[(t1['Date']>='10/01/1990')]]
Q_list = ['Q1', 'Q2', 'Q3', 'Q4']
          
for qi, t in enumerate(t_list):
    print('Running',Q_list[qi])
    X, y = get_Xy(t)
    
    # Station data
    res_df.loc[Q_list[qi]+'_Mon', :] = summary_stats(y)
    
    ## Spatial Averaging
    from polire import SpatialAverage
    spatial = SpatialAverage(coordinate_type='Geographic', radius=16.1)
    spatial.fit(X, y)
    
    spatial_predictions = spatial.predict(centroids)
    spatial_predictions = spatial_predictions[~np.isnan(spatial_predictions)]
    res_df.loc[Q_list[qi]+'_SA', :] = summary_stats(spatial_predictions)
    
    ## IDW
    from polire import IDW
    idw = IDW(coordinate_type='Geographic')
    idw.fit(X, y)
    idw_predictions = idw.predict(centroids)
    res_df.loc[Q_list[qi]+'_IDW', :] = summary_stats(idw_predictions)
    
    ## KNN
    from polire import CustomInterpolator
    from sklearn.neighbors import KNeighborsRegressor
    knn = CustomInterpolator(KNeighborsRegressor(n_neighbors=1))
    knn.fit(X, y)
    knn_predictions = knn.predict(centroids)
    res_df.loc[Q_list[qi]+'_KNN', :] = summary_stats(knn_predictions)

Running Q1
Running Q2
Running Q3
Running Q4


### Kriging is done region wise. 

### Q1, Q2, Q3, Q4

In [18]:
def make_gdf(df, gdf):
    states = df['STATE_CODE'].unique()
    tgdf = gdf[gdf['STATE_FIPS'].isin([resize(x) for x in states])]
    return tgdf

def get_centroids(gdf):
    shapes = gdf['geometry'].unique()
    centroids = [shapes[i].centroid.wkt for i in range(len(shapes))]
    centroids = [centroids[i].split(' ')[1:] for i in range(len(centroids))]
    centroids = [[np.float64(centroids[i][0][1:]), np.float64(centroids[i][1][:-1])] for i in range(len(centroids))]
    centroids = np.array(centroids)
    return centroids

def resize(x):
    if len(str(x))==1:
        return '0'+str(x)
    else:
        return str(x)

In [19]:
def krig_interpolate(df, quarter, centroids, nlags):
    
    if quarter == 1:
        df = df[df['Date']<='03/31/1990']
    elif quarter == 2:
        df = df[(df['Date']>='04/01/1990').values & (df['Date']<='06/30/1990').values]
    elif quarter == 3:
        df = df[(df['Date']>='07/01/1990').values & (df['Date']<='09/30/1990').values]
    else:
        df = df[(df['Date']>='10/01/1990').values ]
        
        
    df = func_select_11(df)
    df.drop(columns=['STATE', 'COUNTY_CODE', 'COUNTY', 'PERCENT_COMPLETE'], inplace=True)
    df = df.groupby('Site ID').mean()
    X = df[['SITE_LONGITUDE', 'SITE_LATITUDE']].values
    y = df[['Daily Mean PM10 Concentration']].values.ravel()
    
    X_krig = X.copy()
    X_test_krig = centroids.copy()
    X_krig[:, 0] = X_krig[:, 0] + 360
    krig = Kriging(variogram_model='spherical', coordinate_type='Geographic', nlags = nlags)
    krig.fit(X_krig, y)
    krig_predictions = np.array(krig.predict(centroids))
    
    return krig_predictions


In [20]:
import geopandas as gpd
gdf = gpd.read_file("../../data/block_groups/USA_Block_Groups.shp")

In [21]:
ne_gdf = make_gdf(nedf, gdf)
ne_centroids = get_centroids(ne_gdf)

nw_gdf = make_gdf(nwdf, gdf)
nw_centroids = get_centroids(nw_gdf)
nw_centroids = nw_centroids[nw_centroids[:, 1]>=34.81]

se_gdf = make_gdf(sedf, gdf)
se_centroids = get_centroids(se_gdf)

socal_gdf = make_gdf(scal, gdf)
socal_centroids = get_centroids(socal_gdf)
socal_centroids = socal_centroids[socal_centroids[:, 1]<=34.81]

mw_gdf = make_gdf(mwdf, gdf)
mw_centroids = get_centroids(mw_gdf)

In [22]:
from polire import Kriging

for qi in range(1,4+1):
    print(Q_list[qi-1])
    ne_q = krig_interpolate(nedf, qi, ne_centroids, nlags = 40)
    nw_q = krig_interpolate(nwdf, qi, nw_centroids, nlags = 17)
    se_q = krig_interpolate(sedf, qi, se_centroids, nlags = 40)
    socal_q = krig_interpolate(scal, qi, socal_centroids, nlags = 17)
    mw_q = krig_interpolate(mwdf, qi, mw_centroids, nlags = 40)
    
    usa_kriging_final_q = np.concatenate([np.array(ne_q), np.array(nw_q), np.array(se_q), np.array(socal_q), np.array(mw_q)])
    
    res_df.loc[Q_list[qi-1]+'_Krig', :] = summary_stats(usa_kriging_final_q)

Q1
Q2
Q3
Q4


# Annual

In [23]:
def compute_krig(df, centroids, nlags):
    annual = df.copy()
    annual = annual.groupby('Site ID').mean()
    annual.drop(columns = ['COUNTY_CODE', 'PERCENT_COMPLETE'], inplace =True)
    X = annual[['SITE_LONGITUDE', 'SITE_LATITUDE']].values
    y = annual[['Daily Mean PM10 Concentration']].values.ravel()
    X[:, 0] = X[:, 0] + 360
    k = Kriging(variogram_model='spherical', coordinate_type='Geographic', nlags=nlags)
    k.fit(X, y)
    krig_predicted = k.predict(centroids)
    return krig_predicted

In [24]:
ne_krig = compute_krig(nedf, ne_centroids, nlags = 40)
nw_krig = compute_krig(nwdf, nw_centroids, nlags = 17)
se_krig = compute_krig(sedf, se_centroids, nlags = 40)
socal_krig = compute_krig(scal, socal_centroids, nlags = 17)
mw_krig = compute_krig(mwdf, mw_centroids, nlags = 40)

usa_kriging_final = np.concatenate([np.array(ne_krig), np.array(nw_krig), np.array(se_krig), np.array(socal_krig), np.array(mw_krig)])

res_df.loc['AA_Krig', :] = summary_stats(usa_kriging_final)

## Final result

In [25]:
print(res_df.to_markdown())

|         |    mean |     25% |     50% |     75% |
|:--------|--------:|--------:|--------:|--------:|
| AA_Mon  | 28.9437 | 22.3917 | 27.2998 | 33.6342 |
| AA_SA   | 31.9448 | 25.8334 | 30.1789 | 34.7331 |
| AA_IDW  | 30.8187 | 26.9154 | 29.4278 | 32.4222 |
| AA_KNN  | 30.3379 | 24.5088 | 28.2745 | 34.0741 |
| Q1_Mon  | 27.4402 | 20.0667 | 25.3786 | 32.6231 |
| Q1_SA   | 30.635  | 23.7462 | 28.9365 | 34.25   |
| Q1_IDW  | 28.8243 | 24.2218 | 26.7869 | 31.0557 |
| Q1_KNN  | 28.4217 | 21.3333 | 26.0667 | 32.6    |
| Q2_Mon  | 26.0074 | 20      | 24.7238 | 30.5333 |
| Q2_SA   | 28.5194 | 23.0333 | 27.2667 | 32.9254 |
| Q2_IDW  | 27.8907 | 24.2632 | 27.1835 | 30.0498 |
| Q2_KNN  | 27.2738 | 21.7333 | 25.8    | 30.4545 |
| Q3_Mon  | 33.5649 | 25.6381 | 33.1875 | 40.4308 |
| Q3_SA   | 36.3301 | 30.1429 | 36      | 40.9667 |
| Q3_IDW  | 36.026  | 31.208  | 35.5538 | 40.4061 |
| Q3_KNN  | 35.5575 | 28.9333 | 34.6    | 40.7333 |
| Q4_Mon  | 28.5522 | 20.0625 | 25.9231 | 32.7692 |
| Q4_SA   | 

In [26]:
print('Done in',(time()-init)/60,'minutes')

Done in 5.478180766105652 minutes
